### General analysis of november data

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
import pandas as pd
from retrieve_data import read_waze_data
import os
import datetime as dt

In [3]:
[df_tt, df_r] = read_waze_data('21.11.2018')

In [4]:
#df_tt.head()

In [5]:
def process_waze_data(df_tt, df_r):

    df_tt['date'] = df_tt['updatetime'].dt.date
    df_tt = df_tt.merge(df_r[['name', 'length']], on='name', how='left')
    df_tt = df_tt.loc[df_tt['length'].isnull() == False, :]
    df_tt['time/length[s/km]'] = (df_tt['time'] / (df_tt['length']))*1000
    df_tt['length/time[km/h]'] = (df_tt['length'] / (df_tt['time']))*3.6

    return df_tt

In [6]:
df_tt = process_waze_data(df_tt, df_r)

In [7]:
#df_tt.head()

Select the dates

In [8]:
start_date = dt.date(2018, 11, 5)
except_7_nov = dt.date(2018,11,7) #energy cut-off
except_12_nov = dt.date(2018,11,12) #hail storm
end_date = dt.date(2018, 11, 18)

In [9]:
mask = (df_tt['date'] >= start_date) & (df_tt['date'] <= end_date) & (df_tt['date'] != except_7_nov) & (df_tt['date'] != except_12_nov)
nov_df = df_tt.loc[mask]
print(len(nov_df.index))

200604


In [10]:
include_24_oct = dt.date(2018,10,24)
include_19_nov = dt.date(2018,11,19)

In [11]:
include_mask = (df_tt['date'] == include_24_oct) | (df_tt['date'] == include_19_nov)
replacing_df = df_tt.loc[include_mask]
print(len(replacing_df.index))

31392


In [12]:
analyze_df = pd.concat([nov_df,replacing_df])
print(len(analyze_df.index))

231996


In [13]:
def day_type(x):
    if (0<=x.weekday()<5):
        return 'L'
    elif (x.weekday()==5):
        return 'S'
    elif (x.weekday()==6):
        return 'D'

In [14]:
analyze_df['day'] = analyze_df['date'].apply(day_type)
analyze_df.sort_values(by=['updatetime'], inplace = True)
analyze_df['updatetime_tmp'] = analyze_df['updatetime'].apply(lambda x: x.replace(month=1,day=1)) #ugly

In [15]:
#analyze_df.head()

In [16]:
grouped_df_tt = analyze_df.groupby(['name', 'length', 'day', pd.Grouper(freq='15min', key='updatetime_tmp')])['time/length[s/km]','length/time[km/h]'].mean()
grouped_df_tt.reset_index(inplace=True)
grouped_df_tt['time'] = grouped_df_tt['updatetime_tmp'].dt.time

A simple test

In [17]:
type_of_days = 3
unique_sections = len(grouped_df_tt['name'].unique())
n_time_windows = 24*60/15

In [18]:
len(grouped_df_tt.index) == type_of_days * unique_sections * n_time_windows

True

Some other things

In [19]:
grouped_df_tt.drop(columns = 'updatetime_tmp', inplace=True)

In [20]:
grouped_df_tt['end_time'] = grouped_df_tt['time'].shift(-1)

In [21]:
#grouped_df_tt.tail()

In [22]:
from utils import constants

In [23]:
travel_times_dir = constants.travel_times_dir

In [24]:
jgibson_path = os.path.join(travel_times_dir,'analysis/mean_travel_times.csv')

In [25]:
grouped_df_tt.to_csv(jgibson_path, sep=';')